In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import pickle
from torchvision.transforms import Compose, ToTensor
import pandas as pd
import plotly.express as px
from tqdm import tqdm

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [4]:
import sys
sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.training import CTSV1
from base_code.plugins import WeightStoragePlugin

# Dataset and definitions

## Preprocessing definitions

In [5]:
train_transform = Compose([
    ToTensor(),
])

test_transform = Compose([
    ToTensor(),
])

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
# mnist_train = MNIST(DATASETS_PATH, train=True, download=True, transform=train_transform)
# mnist_test = MNIST(DATASETS_PATH, train=False, download=True, transform=test_transform)

scenario = PermutedMNIST(10, seed=1234, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
# scenario = nc_benchmark(
#     mnist_train, mnist_test,
#     n_experiences=len(mnist_train.classes), shuffle=True, seed=1234, task_labels=False
# )

train_stream = scenario.train_stream
test_stream = scenario.test_stream

## Evaluation metrics definition

In [18]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    loggers=[InteractiveLogger()]
)

## Plugins definitions

In [19]:
model_plugins = [
    WeightStoragePlugin()
]

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [20]:
# model = MLP(n_classes=scenario.n_classes, n_channels=1, width=28, height=28)
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28*28, hidden_layers=2, hidden_size=100, drop_rate=0)
optimizer = SGD(model.parameters(), lr=1e-3)
criterion = CrossEntropyLoss()
strategy = CTSV1(
    model,
    optimizer,
    criterion,
    lambda_l1=1.0, lambda_l2=1e-2, eps=1e-5,
    train_epochs=5, train_mb_size=256, eval_mb_size=128,
    plugins=model_plugins,
    evaluator=eval_plugin,
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [21]:
results = list()

for experience in tqdm(train_stream):
    strategy.train(experience)
    metrics = strategy.eval(test_stream)
    results.append(metrics)

  0%|          | 0/10 [00:00<?, ?it/s]

-- >> Start of training phase << --
100%|██████████| 235/235 [00:03<00:00, 66.08it/s]
Epoch 0 ended.
100%|██████████| 235/235 [00:03<00:00, 67.31it/s]
Epoch 1 ended.
100%|██████████| 235/235 [00:03<00:00, 66.96it/s]
Epoch 2 ended.
100%|██████████| 235/235 [00:03<00:00, 67.49it/s]
Epoch 3 ended.
100%|██████████| 235/235 [00:03<00:00, 66.35it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 145.15it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.6829
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 145.44it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.1832
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 141.03it/s]
> Eval

 10%|█         | 1/10 [00:23<03:29, 23.22s/it]


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.1006
-- >> End of eval phase << --
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.1751
-- >> Start of training phase << --
100%|██████████| 235/235 [00:03<00:00, 65.25it/s]
Epoch 0 ended.
100%|██████████| 235/235 [00:03<00:00, 63.98it/s]
Epoch 1 ended.
100%|██████████| 235/235 [00:03<00:00, 66.08it/s]
Epoch 2 ended.
100%|██████████| 235/235 [00:03<00:00, 65.92it/s]
Epoch 3 ended.
100%|██████████| 235/235 [00:03<00:00, 66.22it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 145.39it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.5708
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 140.04it/s]
> Eval on experience 1 (Task 

 20%|██        | 2/10 [00:46<03:07, 23.40s/it]

...


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.1431
-- >> End of eval phase << --
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.5772
-- >> Start of training phase << --
100%|██████████| 235/235 [00:03<00:00, 64.73it/s]
Epoch 0 ended.
100%|██████████| 235/235 [00:03<00:00, 66.03it/s]
Epoch 1 ended.
100%|██████████| 235/235 [00:03<00:00, 66.18it/s]
Epoch 2 ended.
100%|██████████| 235/235 [00:03<00:00, 65.95it/s]
Epoch 3 ended.
100%|██████████| 235/235 [00:03<00:00, 66.33it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 146.81it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.4317
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 141.89it/s]
> Eval on experience 1 (Task 

 90%|█████████ | 9/10 [03:29<00:23, 23.27s/it]


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.1655
-- >> End of eval phase << --
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.6180
-- >> Start of training phase << --
100%|██████████| 235/235 [00:03<00:00, 65.85it/s]
Epoch 0 ended.
100%|██████████| 235/235 [00:03<00:00, 66.50it/s]
Epoch 1 ended.
100%|██████████| 235/235 [00:03<00:00, 64.70it/s]
Epoch 2 ended.
100%|██████████| 235/235 [00:03<00:00, 65.56it/s]
Epoch 3 ended.
100%|██████████| 235/235 [00:03<00:00, 62.84it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 142.15it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.3801
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 135.80it/s]
> Eval on experience 1 (Task 

100%|██████████| 10/10 [03:52<00:00, 23.28s/it]


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.8340
-- >> End of eval phase << --
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.6812


In [22]:
accuracies: dict[int, list[float]] = dict()

for i in range(10):
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp00{i}"][1]

accuracies["Overall"] = eval_plugin.get_all_metrics()["Top1_Acc_Stream/eval_phase/test_stream/Task000"][1]

In [23]:
acc_df = pd.DataFrame(accuracies)
acc_df.index = range(10)

In [24]:
# lineplot every task acc per training
fig = px.line(acc_df, x=acc_df.index, y=acc_df.columns, range_y=[0, 1])
fig.show()

# Store metrics

In [26]:
pickle.dump(accuracies, open(SAVED_METRICS_PATH / "cts_v1.pkl", "wb"))